In [1]:
import torch 
import torch.nn.functional as F
from torch import nn

In [25]:
# CenteredLayer类要从其输入中减去均值。 要构建它，只需继承基础层类并实现前向传播功能。

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,X):
        return X-X.mean()
    
layer=CenteredLayer()
X=torch.FloatTensor([1,2,3,4,5])# 均值为3

layer(X) # 验证是否如预期 结果为 3-每一个元素

tensor([-2., -1.,  0.,  1.,  2.])

In [26]:
# 将层作为组件合并到更复杂的模型中。

net=nn.Sequential(nn.Linear(8,128),
                 CenteredLayer()
                 )

print(net)
Y=net(torch.randn(4,8))
Y.mean()


Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)


tensor(-7.4506e-09, grad_fn=<MeanBackward0>)

In [27]:
# 带参数的层 实现自定义版本的全连接层。
# 定义具有参数的层， 这些参数可以通过训练进行调整。

class MyLinear(nn.Module):
    # in_units和units，分别表示输入数和输出数。
    def __init__(self,in_units,units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,units))
        self.bias=nn.Parameter(torch.randn(units,))

        
    def forward(self,X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)
    
linear=MyLinear(5,3)
print(linear)
linear.weight,linear.bias

MyLinear()


(Parameter containing:
 tensor([[ 0.6522,  0.0303,  1.1214],
         [-0.0224,  1.8574,  0.3622],
         [ 1.0422, -0.3849, -0.6989],
         [-1.1314,  1.9091,  0.6286],
         [ 0.0341, -0.3816,  0.8479]], requires_grad=True),
 Parameter containing:
 tensor([ 1.1345, -0.1159,  0.5093], requires_grad=True))

In [28]:
# 自定义层直接执行前向传播计算。
linear(torch.rand(3,5))

tensor([[1.9463, 1.2882, 1.5520],
        [2.0596, 0.3962, 1.2614],
        [0.8961, 2.6895, 2.1734]])

In [35]:
# 使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层
net = nn.Sequential(MyLinear(64,8),
                   MyLinear(8,1))
print(net)
X=torch.rand(2,64)
net(X)

Sequential(
  (0): MyLinear()
  (1): MyLinear()
)


tensor([[2.1738],
        [6.2272]])

我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。

在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。

层可以有局部参数，这些参数可以通过内置函数创建。